## Can a `vectorize` function call a `jit` function?

In [1]:
import numpy
from numba import vectorize, jit, njit

In [2]:
@jit
def add(a, b):
    return a + b

In [3]:
@vectorize
def add_vec(a, b):
    return add(a, b)

In [4]:
to_add = numpy.arange(10000).reshape(5000, 2)

In [14]:
%timeit add_vec(to_add[:, 0], to_add[:, 1])

5.76 µs ± 67.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Can a `jit` function call a `vectorize` function?

In [15]:
@vectorize
def add(a, b):
    return a + b

@jit
def add_vec(a, b):
    return add(a, b)

In [17]:
%timeit add_vec(to_add[:, 0], to_add[:, 1])

4.56 µs ± 79.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In this case the jitted vectorized function is faster than the vectorized jitted function

## How does this compare to using just jit or vectorize?

In [5]:
import numpy as np

In [20]:
@jit
def add_vec(a, b):
    Ar, Ac = a.shape
    c = np.zeros_like(a)
    for row in range(Ar):
        for col in range(Ac):
            c[row,col] = a[row,col] + b[row,col]
    return(c)



In [21]:
add_vec(to_add,to_add)

array([[    0,     2],
       [    4,     6],
       [    8,    10],
       ..., 
       [19988, 19990],
       [19992, 19994],
       [19996, 19998]])

In [22]:
%timeit add_vec(to_add,to_add)

17.9 µs ± 186 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [23]:
@vectorize
def add_vec(a, b):
    return(a+b)

In [24]:
add_vec(to_add,to_add)

array([[    0,     2],
       [    4,     6],
       [    8,    10],
       ..., 
       [19988, 19990],
       [19992, 19994],
       [19996, 19998]])

In [25]:
%timeit add_vec(to_add,to_add)

3.89 µs ± 75.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Depending on what you are jitting and vectorizing, it may be faster to use one or a combination of methods. It depends on the case, with a certian compute overhead from the core function, the number of loops, etc.

## Can I use `isinstance` in `jit`ted functions?  

No, but there is a solution, `generated_jit`

Imagine you have a function that takes an array but wants the scalar sum of that array to operate on.  You can just use `.sum()`.  But sometimes, instead of an array getting passed in, a scalar gets passed, which will throw an error since scalars don't have a `sum` method.  

In [18]:
from numba import types, generated_jit

This is an example of defensive coding

In [19]:
@generated_jit
def safesum(M):
    if isinstance(M, types.Array):
        return lambda M: M.sum()
    else:
        return lambda M: M

In [20]:
safesum(to_add)

49995000

In [21]:
safesum(5)

5

### Note:

* `safesum` gets called with the Numba types of the argument, not their values
* it returns a _function_, not a value.  

You could use this to have different functions called depending on the type of the inputs

In [22]:
def add(a, b):
    return a + b

add_nums = jit(nopython=True)(add)
add_str = jit()(add)

In [23]:
@generated_jit
def safeadd(a, b):
    if isinstance(a, types.Opaque):
        return add_str
    else:
        return add_nums

In [24]:
safeadd('3', '4')

'34'

In [25]:
safeadd(3, 4)

7

# Tips

Set envvar `NUMBA_DISABLE_JIT=1` to disable numba compilation (for debugging)

Install the "Hide Traceback" extension if you're prototyping in a notebook.

# Errors

### Unification errors

Thanks to Graham Markhall for the idea for these examples:
http://gmarkall.github.io/tutorials/pycon-uk-2015

When Numba compiles a function just-in-time, it needs to declare the type of the output(s).  If it can't do that in a consistent way, it gets upset.

In [26]:
@jit(nopython=True)
def get_low(a, b, c):
    if c:
        return a
    else:
        return b

In [27]:
get_low(3., (3, 2), True)

TypingError: Failed at nopython (nopython frontend)
Can't unify return type from the following types: (int64 x 2), float64

In [28]:
get_low(4, numpy.zeros(3), True)

TypingError: Failed at nopython (nopython frontend)
Can't unify return type from the following types: array(float64, 1d, C), int64

In [29]:
get_low(3, 4, True)

3

In [30]:
@jit(nopython=True)
def dont_index_a_scalar(a):
    return a[0]

In [31]:
dont_index_a_scalar(5.)

TypingError: Failed at nopython (nopython frontend)
Invalid usage of getitem with parameters (float64, int64)
 * parameterized
File "<ipython-input-30-ebd0b08b1cdb>", line 3
[1] During: typing of intrinsic-call at <ipython-input-30-ebd0b08b1cdb> (3)
[2] During: typing of static-get-item at <ipython-input-30-ebd0b08b1cdb> (3)

## Globals are treated as compile-time constants by Numba

In [32]:
a = 5

In [33]:
@njit
def add_to_a(b):
    return a + b

In [34]:
add_to_a(7)

12

In [37]:
a = 120

In [38]:
add_to_a(7)

12

### Solution: Don't use globals(!)

Seriously.  Don't.  But if you must, you can force a recompile of the jitted function.

In [39]:
add_to_a.recompile()

In [40]:
add_to_a(7)

127